In [7]:
import requests
from datetime import datetime

'2025-03-15'

In [2]:
response = requests.get("http://127.0.0.1:8000/meals/names")
print(response.status_code, response.text)


200 ["Meal 3","Breakfast","Chicken Vegetables","Isolate Shake","Beef Potato Dinner"]


In [2]:
ollama_url = "http://localhost:11434/api/generate"
data = {"model": "llama3.2", "prompt": "Suggest a meal for 500 kcal and 30g protein.", "stream": False}

response = requests.post(ollama_url, json=data)
print("Status Code:", response.status_code)
print("Response:", response.json())



Status Code: 200
Response: {'model': 'llama3.2', 'created_at': '2025-03-06T20:58:32.3361524Z', 'response': "Here's a suggestion for a meal that provides approximately 500 kcal and 30g of protein:\n\n**Grilled Chicken Breast with Quinoa and Vegetables**\n\n* 1 grilled chicken breast (4 oz, 120g): 140 kcal, 25g protein\n* 1/2 cup cooked quinoa: 100 kcal, 4g protein\n* 1 cup mixed vegetables (such as broccoli, carrots, bell peppers): 50 kcal, 2g protein\n\nTotal calories: approximately 290 kcal\nTotal protein: approximately 31g\n\nTo add some healthy fats and fiber to the meal, you could also include:\n\n* 1 tablespoon olive oil: 120 kcal, negligible protein\n* 1/4 cup chopped nuts or seeds (such as almonds or pumpkin seeds): 100 kcal, 2g protein\n\nTotal calories with added ingredients: approximately 510 kcal\nTotal protein with added ingredients: approximately 33g\n\nThis meal provides a balanced mix of protein, complex carbohydrates, and healthy fats, while staying within the desired c

In [6]:
response = requests.post("http://127.0.0.1:8000/generate_meal/", json={"calories": 500, "protein": 30, "carbs": 50, "fats": 10})

print("Status Code:", response.status_code)
print("Response Text:", response.text)

Status Code: 200
Response Text: {"meals":[{"meal":"Breakfast","foods":["Oats","White potatoes"],"grams":60,"calories":341,"protein":11.3,"carbs":31.9,"fats":1.4},{"meal":"Lunch","foods":["Chicken Breast","White potatoes"],"grams":72,"calories":341,"protein":29.2,"carbs":23.9,"fats":4},{"meal":"Dinner","foods":["Chicken Breast","Oats"],"grams":70,"calories":261,"protein":28.6,"carbs":37.8,"fats":4}]}


In [4]:
### Test API to generate Meal Plan

API_URL = "http://127.0.0.1:8000/generate_meal/"

data = {
    "calories": 500,
    "protein": 30,
    "carbs": 50,
    "fats": 10
}

response = requests.post(API_URL, json=data)

print("Status Code:", response.status_code)
print("Response Text:", response.text)  # Check if it's empty or contains errors


Status Code: 200
Response Text: {"error":"Unexpected error: name 'ollama_response' is not defined"}


In [6]:
response = requests.post("http://127.0.0.1:8000/generate_meal/", json={"calories": 500, "protein": 30, "carbs": 50, "fats": 10})

print("Status Code:", response.status_code)
print("Response Text:", response.text)



Status Code: 500
Response Text: Internal Server Error


In [21]:
import sqlite3

conn = sqlite3.connect("food_macros.db")
cursor = conn.cursor()

# Show table structure
cursor.execute("PRAGMA table_info(foods)")
columns = cursor.fetchall()

print("Current Database Structure:")
for col in columns:
    print(col)

conn.close()


Current Database Structure:
(0, 'name', 'VARCHAR', 1, None, 1)
(1, 'calories', 'FLOAT', 1, None, 0)
(2, 'protein', 'FLOAT', 1, None, 0)
(3, 'carbs', 'FLOAT', 1, None, 0)
(4, 'fats', 'FLOAT', 1, None, 0)


In [1]:
### Updating Database (in this case to remove Fiber)
import sqlite3

# Connect to the database
conn = sqlite3.connect("food_macros.db")
cursor = conn.cursor()

# Step 1: Rename the old table
cursor.execute("ALTER TABLE foods RENAME TO old_foods")

# Step 2: Create a new table without 'fiber'
cursor.execute("""
    CREATE TABLE foods (
        name TEXT PRIMARY KEY,
        calories REAL NOT NULL,
        protein REAL NOT NULL,
        carbs REAL NOT NULL,
        fats REAL NOT NULL
    )
""")

# Step 3: Copy data from old table to new table
cursor.execute("""
    INSERT INTO foods (name, calories, protein, carbs, fats)
    SELECT name, calories, protein, carbs, fats FROM old_foods
""")

# Step 4: Drop the old table
cursor.execute("DROP TABLE old_foods")

# Commit and close
conn.commit()
conn.close()

print("Database updated: 'fiber' column removed.")


Database updated: 'fiber' column removed.


In [22]:
conn = sqlite3.connect("food_macros.db")
cursor = conn.cursor()

# Show final table structure
cursor.execute("PRAGMA table_info(foods)")
columns = cursor.fetchall()

print("Updated Database Structure:")
for col in columns:
    print(col)

conn.close()


Updated Database Structure:
(0, 'name', 'VARCHAR', 1, None, 1)
(1, 'calories', 'FLOAT', 1, None, 0)
(2, 'protein', 'FLOAT', 1, None, 0)
(3, 'carbs', 'FLOAT', 1, None, 0)
(4, 'fats', 'FLOAT', 1, None, 0)


In [3]:
import sqlite3
import pandas as pd


# Connect to the database
conn = sqlite3.connect("food_macros.db")
cursor = conn.cursor()

# Delete meals with specific meal names
cursor.execute("DELETE FROM meals WHERE meal_name IN (?, ?);", ("Bedtime Snack", "Lean Beef Potato Dinner"))

# Commit the changes and close the connection
conn.commit()
conn.close()

print("Deleted all entries with meal_name 'Z'morge Oats' and 'Lean Beef Potato Dinner'.")


Deleted all entries with meal_name 'Z'morge Oats' and 'Lean Beef Potato Dinner'.
